# RNN

### Character based recurrent neural network

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
file_url = 'https://www.gutenberg.org/files/1400/1400-0.txt'
file_path = tf.keras.utils.get_file('1400-0.txt', file_url)

1056768/1049619 [==============================] - 11s 11us/step


In [41]:
text = open(file_path).read()
# Strip off instruction text
story_text = text[824:18781]

In [42]:
# get unique characters
vocab = sorted(set(text))
print(f'Unique characters: {len(vocab)}\n {vocab}')

Unique characters: 88
 ['\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ê', 'ô', '“', '”', '\ufeff']


In [49]:
# Assign index values to each character
char_to_index = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = np.array(vocab)

In [51]:
# translate the text to array of integers

text_as_int = np.asarray([char_to_index[char] for char in text])

In [54]:
text_as_int.shape

(1013445,)

In [64]:
# show character mapping

start, stop = 30, 50

print(text[start:stop])
for i in range(start, stop):
    print(f'{text[i]}:{text_as_int[i]}', end=' ')

f Great Expectations
f:62  :1 G:35 r:74 e:61 a:57 t:76  :1 E:33 x:80 p:72 e:61 c:59 t:76 a:57 t:76 i:65 o:71 n:70 s:75 

#### Prepare an input dataset from the text

In [73]:
# Create input Dataset

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for idx in char_dataset.take(70):
    print(idx_to_char[idx], end= '')

﻿The Project Gutenberg EBook of Great Expectations, by Charles Dickens

In [166]:
# Batch data for training

sequence_len = 80
examples_per_epoch = len(text) / sequence_len

sequences = char_dataset.batch(batch_size=sequence_len + 1, drop_remainder=True)

In [167]:
def split_input_target(text_chunk):
    """
        Creates the input and target data by shifting one
        character to the right
        
        Example: Outside -> utsider
    """
    input_txt = text_chunk[:-1]
    target_txt = text_chunk[1:]
    
    return input_txt, target_txt

In [168]:
dataset = sequences.map(split_input_target)

### Visualization of the data

###### Display the input and target data

- `dataset.take(n)`returns `n` batches.
- The batch size = `sequence_len` characters

In [169]:
# display for 2 batches
input_example, target_example = [], []

for input, target in dataset.take(1):
    print(r'Input: ', ''.join(idx_to_char[input.numpy()]))
    print(r'Target: ', ''.join(idx_to_char[target.numpy()]))
    print('----------------')
    
    input_example.append(input.numpy())
    target_example.append(target.numpy())

Input:  ﻿The Project Gutenberg EBook of Great Expectations, by Charles Dickens

This eBo
Target:  The Project Gutenberg EBook of Great Expectations, by Charles Dickens

This eBoo
----------------


##### Display input and the expected output

In [170]:

for i, (input_idx, target_idx) in enumerate(zip(input_example[0][:5], target_example[0][:5])):
    print(f'step: {i}\n')
    print(f'input: {idx_to_char[input_idx]} ({input_idx})')
    print(f'target:  {idx_to_char[target_idx]} ({target_idx})')
    

step: 0

input: ﻿ (87)
target:  T (48)
step: 1

input: T (48)
target:  h (64)
step: 2

input: h (64)
target:  e (61)
step: 3

input: e (61)
target:    (1)
step: 4

input:   (1)
target:  P (44)


#### Preprocess the data for training

In [171]:
batch_size = 64 # chars in batch
steps_per_epoch = examples_per_epoch // batch_size
buffer_size = text_as_int.size

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

# re-feed data to the model from the beginning
dataset = dataset.repeat()

### Build model

- The model has the layers:

    a) Embedding layers - Lookup table of vectors
    
    b) Gate Recurrent Unit
    
    c) Dense Layer

In [172]:
vocab_len = len(vocab)
embedding_dimension = 256
recurrent_nn_units = 1024

In [173]:
if tf.test.is_gpu_available():
    recurrent_nn = tf.keras.layers.CuDNNGRU
    print('Using GPU')
else:
    from functools import partial
    recurrent_nn = partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
    print('Using CPU')

Using CPU


In [174]:
def build_model():
    layers = [tf.keras.layers.Embedding(input_dim=vocab_len,
                                        output_dim=embedding_dimension,
                                        batch_input_shape=[batch_size, None]
                                       ),
              recurrent_nn(units=recurrent_nn_units,
                          return_sequences=True,
                          stateful=True),
              tf.keras.layers.Dense(vocab_len)
             ]
    model = tf.keras.Sequential(layers)
    
    return model

##### instantiate model

In [175]:
model = build_model()

In [176]:
model

In [180]:
# Check output shape of model

for input_batch, target_batch in dataset.take(1):
    batch_pred = model(input_batch)
    print(f'Output shape: {batch_pred.shape}   # [Batch, sequence_len, vocab_len]')

Output shape: (64, 80, 88)   # [Batch, sequence_len, vocab_len]


In [181]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           22528     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 88)            90200     
Total params: 4,051,032
Trainable params: 4,051,032
Non-trainable params: 0
_________________________________________________________________
